In [ ]:
!nvidia-smi

# Init

In [ ]:
!git clone --depth=1 https://github.com/AdeelH/potsdam-batch-exp.git

In [1]:
BATCH_SIZE = 16
PRETRAINED = True
LAST_CROSS = False

MODEL_ARCH = 'unet'
MODEL_BASE = 'resnet18'
CHANNEL_VARIATION = 'rgb_e'
MODEL_VARIATION = f'lc_{LAST_CROSS}'
MODEL_VARIATION += f'_bnrelu_add'


In [2]:
EXPERIMENT_NAME = f'ss_{CHANNEL_VARIATION}_{MODEL_ARCH}_{MODEL_BASE}{"p" if PRETRAINED else ""}_{MODEL_VARIATION}_bsz_{BATCH_SIZE}_longer_run'
print(EXPERIMENT_NAME)

S3_BUCKET = 'raster-vision-ahassan'
S3_ROOT = f'potsdam/experiments/output/{EXPERIMENT_NAME}'

ss_rgb_e_unet_resnet18p_lc_False_bnrelu_add_bsz_16_longer_run


In [3]:
EPOCHS = 40
LR_START = 1e-1
LR_END = 1e-3

MOMENTUM = 0.9
WEIGHT_DECAY = 5e-4

CHECKPOINT_INTERVAL = 10
BATCH_CB_INTERVAL = 10

In [4]:
import os
from pathlib import Path
import glob
from datetime import datetime

import numpy as np
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
import torch.utils
import torchvision as tv
from torchvision import transforms as tf

from fastai.vision import *
from fastai.metrics import error_rate

import matplotlib.pyplot as plt
%reload_ext autoreload
%autoreload 2
# %matplotlib inline

In [5]:
import sys
sys.path.append('potsdam-batch-exp/')

In [13]:
from utils_ import *
from io_ import *
from data_ import *
from transforms import *
from models import *
from training import *
from visualizations import *
from monitoring import *
%reload_ext autoreload
%autoreload 2

In [7]:
CLASS_NAMES = ['building', 'tree', 'low-vegetation', 'clutter', 'car', 'pavement']
NCLASSES = len(CLASS_NAMES)

In [8]:
io_handler = S3IoHandler(
    local_root=EXPERIMENT_NAME, 
    s3_bucket=S3_BUCKET, 
    s3_root=S3_ROOT
)

# Data

In [8]:
potsdam_dict = io_handler.load_pickled_file('potsdam/data/potsdam.pkl', 'data/potsdam.pkl')

## Prepare datasets

In [9]:
CHANNELS = [ch_R, ch_G, ch_B, ch_E]
CHIP_SIZE = 400
STRIDE = 200
DOWNSAMPLING = 2

TRAIN_SPLIT = 0.85

In [10]:
train_transform, val_transform, x_transform, y_transform = tfs_potsdam(channels=CHANNELS, downsampling=DOWNSAMPLING)

In [11]:
original_ds = Potsdam(potsdam_dict, chip_size=CHIP_SIZE, stride=STRIDE, tf=val_transform)
train_ds    = Potsdam(potsdam_dict, chip_size=CHIP_SIZE, stride=STRIDE, tf=train_transform, x_tf=x_transform, y_tf=y_transform)
val_ds      = Potsdam(potsdam_dict, chip_size=CHIP_SIZE, stride=STRIDE, tf=val_transform  , x_tf=x_transform, y_tf=y_transform)

### Train/val split

In [12]:
train_split_size = int((len(train_ds) * TRAIN_SPLIT) // 1)
val_split_size = len(train_ds) - train_split_size
train_split_size, val_split_size

print('train_split_size', train_split_size)
print('val_split_size', val_split_size)

inds = np.arange(len(train_ds))

train_split_size 12867
val_split_size 2271


### Samplers

In [13]:
train_sampler = torch.utils.data.SubsetRandomSampler(inds[:train_split_size])
val_sampler = torch.utils.data.SubsetRandomSampler(inds[train_split_size:])

assert len(set(train_sampler.indices) & set(val_sampler.indices)) == 0

# Model

Use FastAI to create a UNet from a Resnet18

In [9]:
def _base_model(pretrained=False):
    m = tv.models.resnet18(pretrained=pretrained)
    return m

In [10]:
body = create_body(_base_model, pretrained=PRETRAINED)
model = models.unet.DynamicUnet(body, n_classes=NCLASSES, last_cross=LAST_CROSS).cuda()

In [ ]:
io_handler.load_model_weights(
    model, 
    s3_path='potsdam/experiments/output/ss_rgb_unet_resnet18p_lc_False_bsz_16/best_model/best_acc', 
    tgt_path='rgb_model'
)

In [14]:
model[0][0] = ModifiedConv_alt_add(model[0][0], model[0][1], new_conv_in_channels=1).cuda()

In [16]:
# model[0][1] = nn.Identity().cuda()
# model[0][2] = nn.Identity().cuda()
# # model

In [17]:
freeze(model[0][0].original_conv[0])

In [ ]:
# model.eval()
# with torch.no_grad():
#     out = model(torch.rand(4, 4, 200, 200).cuda())
#     print(out.shape)

# Train

## Training monitoring callbacks

In [ ]:

def get_epoch_monitor(io_handler, chkpt_interval=1, viz_root='visualizations/per_epoch'):
    assert chkpt_interval > 0

    filter_path = f'{viz_root}/conv'
    os.makedirs(io_handler.to_local_path(filter_path), exist_ok=True)

    def _monitor(model, optimizer, sched, logs):
        epoch = logs['epoch'][-1]

        track_best_model(io_handler, model, logs)

        if epoch % chkpt_interval == 0:
            io_handler.save_checkpoint(model, optimizer, sched, f'checkpoints/epoch_%04d' % (epoch), info=logs)

        log_str = logs_to_str(logs)
        print(log_str)

        io_handler.save_log('logs.pkl', logs)
        io_handler.save_log_str(f'logs.txt', log_str)
        
        make_plots(io_handler, logs)

        title = 'epoch %04d' % (epoch)
        fs = model[0][0].new_conv[0].weight
        fig = viz_conv_layer_filters(fs.data, title=title, scale_each=False, padding=1)
        io_handler.save_img(fig, f'{filter_path}/epoch_%04d' % (epoch))
        plt.close(fig)

    return _monitor

def get_batch_monitor(io_handler, viz_root='visualizations/per_batch', interval=4):

    filter_path = f'{viz_root}/conv'
    grad_path = f'{viz_root}/conv_grad'
    os.makedirs(io_handler.to_local_path(filter_path), exist_ok=True)
    os.makedirs(io_handler.to_local_path(grad_path), exist_ok=True)

    def _monitor(model, epoch, batch_idx, batch, labels):
        if batch_idx % interval != 0:
            return

        fs = model[0][0].new_conv[0].weight

        title = 'epoch %04d, batch %05d' % (epoch, batch_idx)
        fig = viz_conv_layer_filters(fs.data, title=title, scale_each=False, padding=1)
        io_handler.save_img(fig, f'{filter_path}/epoch_%04d_batch_%05d' % (epoch, batch_idx))
        plt.close(fig)

        fig = viz_conv_layer_filters(fs.grad.data, title=title, scale_each=False, padding=1)
        io_handler.save_img(fig, f'{grad_path}/epoch_%04d_batch_%05d' % (epoch, batch_idx))
        plt.close(fig)

    return _monitor


In [ ]:
epoch_callback = get_epoch_monitor(io_handler, chkpt_interval=CHECKPOINT_INTERVAL)
batch_callback = get_batch_monitor(io_handler, interval=BATCH_CB_INTERVAL)

In [ ]:
train_params = {}
train_params['batch_size'] = BATCH_SIZE
train_params['val_batch_size'] = BATCH_SIZE

In [ ]:
train_dl = torch.utils.data.DataLoader(train_ds, sampler=train_sampler, batch_size=train_params['batch_size']    , pin_memory=False)
val_dl   = torch.utils.data.DataLoader(val_ds  , sampler=val_sampler  , batch_size=train_params['val_batch_size'], pin_memory=False)

In [ ]:
train_params = {}
train_params['epochs'] = EPOCHS
train_params['learning_rate'] = LR_START
train_params['learning_rate_min'] = LR_END

optimizer = optim.SGD(model.parameters(), lr=train_params['learning_rate'], momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
sched = optim.lr_scheduler.CosineAnnealingLR(optimizer, train_params['epochs'], eta_min=train_params['learning_rate_min'])


In [ ]:
from collections import defaultdict

if io_handler.checkpoint_exists():
    logs = restore_training_state(io_handler, model, optimizer, sched, train_params)
else:
    logs = defaultdict(list)

print(logs)

In [ ]:
plt.ioff()
logs = train_seg(model, train_dl, val_dl, optimizer, sched, train_params, 
          epoch_callback=epoch_callback, batch_callback=batch_callback, logs=logs)

### Train some more

In [ ]:
train_params = {}
train_params['epochs'] = EPOCHS
train_params['learning_rate'] = LR_END
train_params['learning_rate_min'] = LR_END / 100
train_params['last_epoch'] = EPOCHS - 1

sched = optim.lr_scheduler.CosineAnnealingLR(optimizer, train_params['epochs'], eta_min=train_params['learning_rate_min'])

plt.ioff()
train_seg(model, train_dl, val_dl, optimizer, sched, train_params, 
          epoch_callback=epoch_callback, batch_callback=batch_callback, logs=logs)

# Create animations

In [ ]:
!apt update -y
!apt install ffmpeg -y

In [ ]:
!ffmpeg -framerate 60 -pattern_type glob -i "$EXPERIMENT_NAME/visualizations/per_batch/conv/*.png" -c:v libx264 \
    -pix_fmt yuv420p "$EXPERIMENT_NAME/visualizations/per_batch/conv/conv.mp4" -y;
!ffmpeg -framerate 60 -pattern_type glob -i "$EXPERIMENT_NAME/visualizations/per_batch/conv_grad/*.png" -c:v libx264 \
    -pix_fmt yuv420p "$EXPERIMENT_NAME/visualizations/per_batch/conv_grad/conv_grad.mp4" -y;

In [ ]:
io_handler.upload_file('visualizations/per_batch/conv/conv.mp4', 'visualizations/per_batch/conv.mp4')
io_handler.upload_file('visualizations/per_batch/conv_grad/conv_grad.mp4', 'visualizations/per_batch/conv_grad.mp4')